<a href="https://colab.research.google.com/github/kareem-project/Dissertation-Project/blob/main/rnn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from google.colab.patches import cv2_imshow

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Dissertation/sudoku.csv", sep=',')

TypeError: ignored

In [ ]:
#Auxiliary Function Used
#Verma, S. (2022). shivaverma/Sudoku-Solver. [online] GitHub. Available at: https://github.com/shivaverma/Sudoku-Solver [Accessed 4 May 2023].

from sklearn.model_selection import train_test_split
import torch.utils.data as data
import torch
def get_data(file):

    data = pd.read_csv(file)

    feat_raw = data['quizzes']
    label_raw = data['solutions']

    feat = []
    label = []

    for i in feat_raw:

        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)

    feat = np.array(feat)
    feat = feat/9
    feat -= .5

    for i in label_raw:

        x = np.array([int(j) for j in i]).reshape((81, 1)) - 1
        label.append(x)

    label = np.array(label)
    del(feat_raw)
    del(label_raw)

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.2, random_state=42)

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_data("/content/drive/MyDrive/Dissertation/sudoku.csv")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Reshape

model = Sequential()

model.add(layers.LSTM(input_shape = (9,9), units = 81))

model.add(layers.Flatten())
model.add(layers.Reshape((81,1)))
model.add(layers.Dense(9))
model.add(layers.Dense(81*9))
model.add(layers.Dense(9, activation = 'softmax'))

model.add(layers.Dropout(0.1))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 81)                29484     
                                                                 
 flatten (Flatten)           (None, 81)                0         
                                                                 
 reshape (Reshape)           (None, 81, 1)             0         
                                                                 
 dense (Dense)               (None, 81, 9)             18        
                                                                 
 dense_1 (Dense)             (None, 81, 729)           7290      
                                                                 
 dense_2 (Dense)             (None, 81, 9)             6570      
                                                                 
 dropout (Dropout)           (None, 81, 9)             0

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)

In [ ]:
model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 128, epochs=10)

Epoch 1/10
6250/6250 [==============================] - 39s 5ms/step - loss: 2.1013 - accuracy: 0.2390 - val_loss: 2.0320 - val_accuracy: 0.3336
Epoch 2/10
6250/6250 [==============================] - 32s 5ms/step - loss: 2.0151 - accuracy: 0.3206 - val_loss: 1.9952 - val_accuracy: 0.3515
Epoch 3/10
6250/6250 [==============================] - 32s 5ms/step - loss: 1.9859 - accuracy: 0.3440 - val_loss: 1.9724 - val_accuracy: 0.3725
Epoch 4/10
6250/6250 [==============================] - 32s 5ms/step - loss: 1.9697 - accuracy: 0.3543 - val_loss: 1.9582 - val_accuracy: 0.3843
Epoch 5/10
6250/6250 [==============================] - 32s 5ms/step - loss: 1.9679 - accuracy: 0.3542 - val_loss: 1.9613 - val_accuracy: 0.3812
Epoch 6/10
6250/6250 [==============================] - 32s 5ms/step - loss: 1.9691 - accuracy: 0.3526 - val_loss: 1.9577 - val_accuracy: 0.3885
Epoch 7/10
6250/6250 [==============================] - 32s 5ms/step - loss: 1.9646 - accuracy: 0.3565 - val_loss: 1.9576 - val_ac

In [ ]:
model.save("my_model_RNN.h5", include_optimizer=True)
